How I plan to start this off. (Jean)

In [2]:
# Just took whatever libraries he put on from class notes so I don't go back and forth. Will change later (maybe?)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [3]:
#Load Data
train_df = pd.read_csv("train.csv.zip")
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
# Use the Count Vectorizer to grab the info and Features I would need. 
vec_cv = CountVectorizer(max_features=150)
tmp = vec_cv.fit_transform(train_df["comment_text"])
tok_cols = vec_cv.get_feature_names_out()
tok_df = pd.DataFrame(tmp.toarray(), columns=tok_cols)
print("original:", train_df["comment_text"].shape)
print("vectorized:", tmp.shape)
tok_df.sample(10)

original: (159571,)
vectorized: (159571, 150)


,about,add,after,again,all,also,am,an,and,any,...,who,why,wikipedia,will,with,work,would,wp,you,your
120760,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54598,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,3
16907,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,2
158649,0,0,0,0,0,0,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4001,0,0,0,0,0,1,0,1,3,0,...,0,0,0,0,0,0,0,0,0,0
72015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
98060,0,0,0,0,0,0,0,0,0,0,...,1,2,0,0,2,0,0,0,3,0
138605,0,1,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
105681,0,0,0,0,0,0,0,1,1,1,...,0,1,0,0,0,0,0,0,2,0
47906,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,2,0


In [5]:
#TF-IDF
vec_tf = TfidfVectorizer(max_features=150, strip_accents="unicode")
tmp = vec_tf.fit_transform(train_df["comment_text"])
tok_cols = vec_tf.get_feature_names_out()
tok_df = pd.DataFrame(tmp.toarray(), columns=tok_cols)
print("original:", train_df["comment_text"].shape)
print("vectorized:", tmp.shape)
tok_df.sample(10)

original: (159571,)
vectorized: (159571, 150)


,about,add,after,again,all,also,am,an,and,any,...,who,why,wikipedia,will,with,work,would,wp,you,your
24945,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.156375,0.000000,...,0.00000,0.319735,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.324259,0.000000
43788,0.000000,0.0,0.000000,0.000000,0.00000,0.254209,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
112746,0.156655,0.0,0.000000,0.217297,0.00000,0.000000,0.000000,0.148222,0.093277,0.000000,...,0.18305,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.096709,0.140441
17633,0.185748,0.0,0.135221,0.128826,0.00000,0.000000,0.000000,0.000000,0.110599,0.000000,...,0.00000,0.000000,0.0,0.000000,0.160083,0.000000,0.0,0.0,0.229339,0.000000
60127,0.000000,0.0,0.000000,0.000000,0.14007,0.000000,0.000000,0.062928,0.118802,0.000000,...,0.00000,0.000000,0.0,0.070186,0.057318,0.000000,0.0,0.0,0.041058,0.059624
73597,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.147599,0.000000,...,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.222231
24154,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
120464,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.359217,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.364299,0.000000
82180,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.215752,0.000000,...,0.00000,0.000000,0.0,0.000000,0.000000,0.263602,0.0,0.0,0.223692,0.324845
139121,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.224452,0.000000,0.159008,0.098715,...,0.00000,0.108373,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.054953,0.000000


I'm comparing count vectorization and TF-IDF. Most likely going to use some sort of vectorization parameters. Most likely would be making a tokenizaer to figure out whether it is a threat or not.